#### Directories etc

In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from Bio import Entrez, SeqIO, AlignIO, pairwise2, Align, Seq, motifs
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from scipy.stats import binom
import math
import random
from tqdm import tqdm
from Comparative_Analysis import Utilities as util
import random
from joblib import Parallel, delayed
from Comparative_Analysis import Blast_Functions as blastfn
import time
import os
import copy
from scipy.stats import chi2
from Comparative_Analysis import ORF_Functions as orffn
from Comparative_Analysis import Alignment as alignfn
from random import sample
from Bio.Align.Applications import MuscleCommandline
import subprocess
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
project_dir = 'F:/Project_Data/Intergenic_Region_Comparative_Analysis'
seq_dir_1 = 'F:/Datasets/NCBI_Refseq_Mycobacteriaceae_All_Levels/ncbi_dataset/data'
seq_dir_2 = 'F:/Datasets/NCBI_Refseq_Actinobacteria_All_Levels/data'
sequence_dirs_1 = util.list_dirs(seq_dir_1)
sequence_dirs_2 = util.list_dirs(seq_dir_2)
tb_species = 'NC_000962.3' 
tb_annotation_dirname = 'GCF_000195955.2'
num_cores = 16
core_numbers = list(range(1, num_cores+1))
muscle_exe = 'C:/Users/nicho/Muscle/muscle3.8.31_i86win32.exe'
full_build = False

#### Create file with all CDS for species and create BLAST databases for TB CDS and All CDS (to do reciprocal best hits)

In [19]:
def generate_protein_dataset(num_subsets, subset_num, dir_list, seqdir): 
    sequence_dirs = util.chunk_list(dir_list, num_subsets, subset_num)
    all_cds = []
    all_tb_cds = []
    names = []
    sequences = []
    locations = []
    for dirname in (sequence_dirs):
        if not(os.path.exists(seqdir + '/' + dirname + '/genomic.gbff')):
            continue
        for genome_record in SeqIO.parse(seqdir + '/' + dirname + '/genomic.gbff', "genbank"):
            accession_ver = genome_record.annotations['accessions'][0] + '.' + str(genome_record.annotations['sequence_version'])
            names.append([accession_ver, genome_record.annotations['organism']])
            full_sequence = str(genome_record.seq)
            sequences.append([accession_ver, full_sequence])
            for feature in genome_record.features:
                a = feature.qualifiers
                if feature.type == 'CDS' and a.get("translation") != None:
                    locus_tag = a.get("locus_tag")[0]
                    accession_locus = accession_ver + '@' + locus_tag
                    translation = a.get("translation")[0]
                    (start, stop, strand) = (int(feature.location.start), int(feature.location.end), int(feature.location.strand))
                    locations.append([accession_locus, (start, stop, strand)])
                    all_cds.append([accession_locus, translation])
                    if dirname == tb_annotation_dirname:
                        all_tb_cds.append([accession_locus, translation])
    return (all_cds, all_tb_cds, names, locations, sequences)           

In [20]:
parallel_output = Parallel(n_jobs=-1)(delayed(generate_protein_dataset)(num_cores, core_number, sequence_dirs_1, seq_dir_1) for core_number in core_numbers)
names_dict_1 = {}
locations_dict_1 = {}
sequence_dict_1 = {}
all_cds_1 = []
all_tb_cds_1 = []
for x in parallel_output:
    all_cds_1 += x[0]
    all_tb_cds_1 += x[1]
    for temp in x[2]:
        names_dict_1[temp[0]] = temp[1]
    for temp in x[3]:
        locations_dict_1[temp[0]] = temp[1]
    for temp in x[4]:
        sequence_dict_1[temp[0]] = temp[1]
with open(project_dir + '/names_dict_1.pkl', 'wb') as f:
        pickle.dump(names_dict_1, f) 

In [21]:
parallel_output = Parallel(n_jobs=-1)(delayed(generate_protein_dataset)(num_cores, core_number, sequence_dirs_2, seq_dir_2) for core_number in core_numbers)
names_dict_2 = {}
locations_dict_2 = {}
sequence_dict_2 = {}
all_cds_2 = []
all_tb_cds_2 = []
for x in parallel_output:
    all_cds_2 += x[0]
    all_tb_cds_2 += x[1]
    for temp in x[2]:
        names_dict_2[temp[0]] = temp[1]
    for temp in x[3]:
        locations_dict_2[temp[0]] = temp[1]
    for temp in x[4]:
        sequence_dict_2[temp[0]] = temp[1]
with open(project_dir + '/names_dict_2.pkl', 'wb') as f:
        pickle.dump(names_dict_2, f) 

In [31]:
if full_build == True:
    util.produce_fasta_file(all_cds_1, project_dir + '/all_cds.fasta')
    util.produce_fasta_file(all_tb_cds_1, project_dir + '/all_tb_cds.fasta')
    blastfn.build_blast_db(project_dir, 'all_cds.fasta', 'Mycobacteriaceae_Refseq', 'F:/Datasets/BLAST/Mycobacteriaceae_Refseq')
    blastfn.build_blast_db(project_dir, 'all_tb_cds.fasta', 'all_tb_cds', 'F:/Datasets/BLAST/all_tb_cds')

In [154]:
if full_build == True:
    util.produce_fasta_file(all_cds_2, project_dir + '/all_cds_2.fasta')
    util.produce_fasta_file(all_tb_cds_2, project_dir + '/all_tb_cds_2.fasta')
    blastfn.build_blast_db(project_dir, 'all_cds_2.fasta', 'Actinobacteria_Refseq', 'F:/Datasets/BLAST/Actinobacteria_Refseq')
    blastfn.build_blast_db(project_dir, 'all_tb_cds_2.fasta', 'all_tb_cds_2', 'F:/Datasets/BLAST/all_tb_cds_2')

100%|██████████| 3906/3906 [00:00<00:00, 50716.03it/s]


In [9]:
if full_build == True:
    blastfn.run_blastp('F:/Datasets/BLAST/Mycobacteriaceae_Refseq', 'all_tb_cds.fasta', 'Mycobacteriaceae_Refseq', 'all_tb_cds_hits.csv', e_value = 1e-10)
    blastfn.run_blastp('F:/Datasets/BLAST/all_tb_cds', 'all_cds.fasta', 'all_tb_cds', 'reverse_hits.csv', e_value = 1e-10)
    a = blastfn.process_blast_output('F:/Datasets/BLAST/Mycobacteriaceae_Refseq/all_tb_cds_hits.csv', names_dict, top_hit_only = False)
    a = a.loc[a.groupby(['query_ref','target_species_name'])['bit_score'].idxmax()]     # Utility top hit method uses accession ver which can differ if multiple sets exist per species in fragmented annotations
    b = blastfn.process_blast_output('F:/Datasets/BLAST/all_tb_cds/reverse_hits.csv', names_dict, top_hit_only = False)
    b = b.loc[b.groupby(['query_ref','target_species_name'])['bit_score'].idxmax()] 
    rbh =  blastfn.keep_reciprocal_best_hits(a, b)
    rbh.to_csv(project_dir + '/reciprocal_best_hits.csv')
else:
    rbh = pd.read_csv(project_dir + '/reciprocal_best_hits.csv')

In [10]:
#if 1==1:
if full_build == True:
    #blastfn.run_blastp('F:/Datasets/BLAST/Actinobacteria_Refseq', 'all_tb_cds_2.fasta', 'Actinobacteria_Refseq', 'all_tb_cds_hits_2.csv', e_value = 1e-10)
    #blastfn.run_blastp('F:/Datasets/BLAST/all_tb_cds_2', 'all_cds_2.fasta', 'all_tb_cds_2', 'reverse_hits_2.csv', e_value = 1e-10)
    a = blastfn.process_blast_output('F:/Datasets/BLAST/Actinobacteria_Refseq/all_tb_cds_hits_2.csv', names_dict_2, top_hit_only = False)
    a = a.loc[a.groupby(['query_ref','target_species_name'])['bit_score'].idxmax()]     # Utility top hit method uses accession ver which can differ if multiple sets exist per species in fragmented annotations
    b = blastfn.process_blast_output('F:/Datasets/BLAST/all_tb_cds_2/reverse_hits_2.csv', names_dict_2, top_hit_only = False)
    b = b.loc[b.groupby(['query_ref','target_species_name'])['bit_score'].idxmax()] 
    rbh_2 =  blastfn.keep_reciprocal_best_hits(a, b)
    rbh_2.to_csv(project_dir + '/reciprocal_best_hits_2.csv')
else:
    rbh_2 = pd.read_csv(project_dir + '/reciprocal_best_hits_2.csv')

#### Function to generate FASTA file containing intergenic regions in orthologous species and run Muscle / R-scape

In [11]:
def generate_alignment(locus, offset, comparison_set):
    target_locus = tb_species+ '@' + locus
    
    results_dir = project_dir + '/' + locus
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    
    if comparison_set == 1:     #1 = Mycobacteria, 2 = Actinobacteria
        rbh = pd.read_csv(project_dir + '/reciprocal_best_hits.csv')
        locations_dict = locations_dict_1
        sequence_dict = sequence_dict_1
    else:
        rbh = pd.read_csv(project_dir + '/reciprocal_best_hits_2.csv')
        locations_dict = locations_dict_2
        sequence_dict = sequence_dict_2
    
    location = locations_dict[target_locus] 
    rbh = rbh[rbh['query_ref'] == target_locus]
    rbh['target_loc'] = rbh['target_ref'].map(locations_dict)
    rbh['cds_count'] = rbh.groupby('target_species_name')['query_ref'].transform('size')
    rbh['min_pct_id'] = rbh.groupby('target_species_name')['percent_identical_matches'].transform('min')
    
    rbh = rbh[rbh['cds_count'] == 1]
    for i, r in rbh.iterrows():
        (start, stop, strand) = r['target_loc']
        rbh.at[i,'start'] = start
        rbh.at[i,'stop'] =  stop
        rbh.at[i,'strand'] = strand
    rbh = rbh[rbh['min_pct_id'] >= 60]
    
    intergenic_regions = []
    
    for i, r in rbh.iterrows():
        target_species = r['target_species_name']
        target_species_accession = r['target_species']
        coords = r['target_loc']
        if coords[2] == 1:
            intergenic_region = (coords[0] - offset, coords[0] + 3)
            intergenic_sequence = sequence_dict[target_species_accession][intergenic_region[0]: intergenic_region[1]]
        else:
            intergenic_region = (coords[1] - 3, coords[1] + offset)
            intergenic_sequence = util.reverse_complement(sequence_dict[target_species_accession][intergenic_region[0]: intergenic_region[1]])

        intergenic_regions.append([target_species.replace(" ", "_"), intergenic_sequence])


    util.produce_fasta_file(intergenic_regions, results_dir +'/intergenic_regions_'+str(comparison_set)+'.fasta')
    cline = MuscleCommandline(muscle_exe, input= results_dir +'/intergenic_regions_'+str(comparison_set)+'.fasta', out = results_dir + '/upstream_region_'+locus + '_alignment_'+str(comparison_set)+'.fasta')
    exception = 0
    try:
        stdout, stderr = cline()
    except Exception as e:
        exception == 1
    if exception == 0:
        blastfn.convert_fasta_to_stockholm(results_dir, 'upstream_region_'+locus + '_alignment_'+str(comparison_set)+'.fasta', 'upstream_region_'+locus + '_alignment_'+str(comparison_set)+'.sto')
        blastfn.run_rscape(results_dir, 'upstream_region_'+locus + '_alignment_'+str(comparison_set)+'.sto', 'rscape_output_'+str(comparison_set))


#### Subset based on loci of interest, and filter orthologues based on percent identity, synteny

In [13]:
tb_loci = []
for genome_record in SeqIO.parse(seq_dir_1 + '/' + tb_annotation_dirname + '/genomic.gbff', "genbank"):
        for feature in genome_record.features:
                a = feature.qualifiers
                if feature.type == 'CDS':
                    tb_loci.append(a.get("locus_tag")[0])

In [ ]:
dont_start = 1
for n, locus in tqdm(enumerate(tb_loci)):
    if locus == 'Rv2312':
        dont_start = 0
    if dont_start == 1:
        continue
    target_locus = tb_species+ '@' + locus 
    location = locations_dict_1[target_locus] 
    if n == 0:
        continue
    if location[2] == 1:
        upstream_cds_stop = locations_dict_1[tb_species + '@' + tb_loci[n-1]][1]
        offset = location[0] - upstream_cds_stop
    else:
        upstream_cds_stop = locations_dict_1[tb_species + '@' + tb_loci[n+1]][0]
        offset = upstream_cds_stop - location[1]

    if offset < 30:
        continue
    else:
        print(locus)
        generate_alignment(locus, offset, 1)
        generate_alignment(locus, offset, 2)

0it [00:00, ?it/s]

Rv2312



100%|██████████| 3/3 [00:00<00:00, 5989.01it/s]

100%|██████████| 3/3 [00:00<00:00, 6000.43it/s]
2301it [00:21, 105.88it/s]

Rv2315c



100%|██████████| 211/211 [00:00<00:00, 422635.22it/s]
2301it [00:36, 105.88it/s]
100%|██████████| 500/500 [00:00<00:00, 250705.56it/s]
2304it [00:50, 36.31it/s] 

...


100%|██████████| 208/208 [00:00<00:00, 104007.54it/s]

100%|██████████| 130/130 [00:00<00:00, 129823.70it/s]
2467it [3:59:17, 23.81s/it]

Rv2477c



100%|██████████| 215/215 [00:00<00:00, 143252.64it/s]

100%|██████████| 500/500 [00:00<00:00, 124994.16it/s]
2468it [4:00:41, 33.55s/it]

Rv2478c



100%|██████████| 162/162 [00:00<00:00, 9814.92it/s]

100%|██████████| 101/101 [00:00<00:00, 9619.53it/s]


#### Extract covariation information to identify potentially significant regions

In [27]:
seq_ids = util.list_dirs(project_dir)
out_list = []
for id in seq_ids:
    if os.path.exists(project_dir + '/' + str(id) + '/rscape_output_2.cov'):
        alignment = util.read_fasta_to_array(project_dir + '/' + str(id) + '/upstream_region_' + str(id) + '_alignment_2.fasta')
        sequences_in_alignment = len(alignment[0])
        alignment_length = len(alignment[1][0])
        with open(project_dir + '/' + str(id) + '/rscape_output_2.cov', 'r') as f:  
            num_pairs = 0
            near_end_covariation = 0
            e_values = []
            for l in f:
                if (not ('#' in l)):
                    a = l.split()
                    if len(a) > 6 and abs(int(a[2])-int(a[1])) > 3:    # Exclude covarying pairs less than 3 bp apart)
                        e_values.append(float(a[4]))
                        num_pairs +=1
                    if len(a) > 6 and abs(int(a[1]) - alignment_length) < 50 and abs(int(a[2]) - alignment_length) < 50:
                        near_end_covariation +=1
        if len(e_values) > 0:
            tot = sum([math.log(x) for x  in e_values])
            num = len(e_values)
            combined_e_value = 1-chi2.cdf(-2*tot, 2*num)
        else:
            combined_e_value = 999
        if combined_e_value < 1e-10:
            print(id, 'sequences=', sequences_in_alignment, 'num_pairs=' ,num_pairs, combined_e_value, alignment_length, near_end_covariation)

Rv1133c sequences= 277 num_pairs= 35 0.0 1550 0
Rv0002 sequences= 214 num_pairs= 9 1.887379141862766e-15 1403 2
Rv0009 sequences= 499 num_pairs= 23 0.0 348 22
Rv0020c sequences= 297 num_pairs= 60 0.0 1458 0
Rv0041 sequences= 500 num_pairs= 44 0.0 656 42
Rv0053 sequences= 500 num_pairs= 18 0.0 811 4
Rv0129c sequences= 123 num_pairs= 9 9.769962616701378e-15 547 0
Rv0154c sequences= 323 num_pairs= 9 0.0 1451 0
Rv0189c sequences= 499 num_pairs= 19 0.0 324 0
Rv0211 sequences= 484 num_pairs= 14 0.0 630 1
Rv0237 sequences= 124 num_pairs= 14 0.0 195 14
Rv0244c sequences= 369 num_pairs= 37 0.0 1119 1
Rv0350 sequences= 486 num_pairs= 11 0.0 691 1
Rv0357c sequences= 496 num_pairs= 11 0.0 311 8
Rv0363c sequences= 497 num_pairs= 7 2.1495027979767656e-12 229 5
Rv0423c sequences= 496 num_pairs= 8 0.0 526 0
Rv0440 sequences= 496 num_pairs= 28 0.0 924 4
Rv0462 sequences= 494 num_pairs= 3 1.7658097206663115e-12 130 3
Rv0468 sequences= 408 num_pairs= 12 1.1102230246251565e-16 307 12
Rv0491 sequences= 499